In [1]:
# import libraries
import hashlib
import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
import datetime
import collections

# following imports are required by PKI
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

In [2]:
class Client:
   def __init__(self):
      random = Crypto.Random.new().read
      self._private_key = RSA.generate(1024, random)
      self._public_key = self._private_key.publickey()
      self._signer = PKCS1_v1_5.new(self._private_key)

   @property
   def identity(self):
      return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

In [3]:
# How to use the client
michael = Client()
print(michael.identity)

30819f300d06092a864886f70d010101050003818d0030818902818100a038ebf7468dcc4950928eb72b1d7f42527b1a9b3a88e9d988c02f56feb7c4af3c7b91947d26bf1aff52a0d670c3df8e29203003db02b3bfb8b7ea10cc7027a7d3169ecf5bf52de015a24929a6253830280c6f85e8e1a5817fd60df50812028a82faadd25139e43ac4bc25f0a2c3b4e4263af49ec691d49f90a34b605ffa46fd0203010001


In [4]:
class Transaction:
    '''
    A class to represent a transaction

    ...
    
    Attributes
    -----------
    sender : str
        The senders public key taken from their instance of the Client()
    recipient : str
        The recipients public key taken from their instance of the Client()
    value : int
        The amount of coins to be sent in the transaction
    time : datetime
        Time the transaction took place

    Mehtods
    -------
    to_dict():
        Utiltiy method that combines all four attributes into a dictionary object
    sign_transaction():
        Used to sign the dictionary object from to_dict with the private key of the sender
    '''
    def __init__(self, sender, recipient, value):
        '''
        Constructs the 4 needed attributes for the person object

            Parameters:
            -----------
                sender : str
                    The senders public key taken from their instance of the Client()
                recipient : str
                    The recipients public key taken from their instance of the Client()
                value : int
                    The amount of coins to be sent in the transaction
        
            Returns:
            --------
                instance variables that can be used by other methods.
        '''
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()

    def to_dict(self):
        '''
        Utiltiy method that combines all four attributes into a dictionary object

        The if, else checks whehter the sender of the transaction has the identity of Genesis. 
        If so they are assigned the value of genesis so their identity is kept secret

        Returns
        -------
        Dictionary Object containing the four attributes that make up a transaction
        '''
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity

        return collections.OrderedDict({
            'sender': identity,
            'recipient': self.recipient,
            'value': self.value,
            'time' : self.time})

     # Used to retrieve the senders private key       
    def sign_transaction(self):
        '''
        Used to sign the dictionary object from to_dict with the private key of the sender

        Returns
        -------
        The private key of the sender
        '''
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')


In [5]:
# Testing the Transaction Class

# Michael is going to send 5 coins to Tyler
# To do this we first create the clients
michael = Client()
tyler = Client()

# Create the transaction with the following code
t = Transaction(
    michael,  # Sender
    tyler.identity, # Gets tylers public key
    5.0  # Amount transferred
)

# Printing the transaction to review it
print(t.to_dict())

# Once we have created the transaction we can sign it with the senders private key.
signature = t.sign_transaction()
print(signature)


OrderedDict([('sender', '30819f300d06092a864886f70d010101050003818d0030818902818100d00bc7114adc5a9f8274974daa283e4cfc1102aaa6a7b834a45ba0aa761ab999d05b165ba335ce52ff56029d6bf89c119f8265c44178a70b09f6cf128f6d2b69cd805a6ce5692ba087d5c5e7a231a33f2fc47988682e50b766e4e8bd80d73cdb7cc4edc9a15dff1ec5cc4d3df3c3278bbdb552104855d71e4b399300e50ef95d0203010001'), ('recipient', '30819f300d06092a864886f70d010101050003818d0030818902818100c13d92946185e5d87c7cf78e6a7a5309da9e5c4c6bd626b4e55f9109542029a3c75d89d3e8a9dd30f8133cb58df26840bed29b3c3bd7b6a14d051f805db205ef8672af04f2b49cb5584af994e1246e32e1abc1020c65886f46b0ea51af2a78a6503ec020ca457b27c9638c421d9570335a28eecde77e0a696665b4894e879ae50203010001'), ('value', 5.0), ('time', datetime.datetime(2021, 2, 4, 4, 20, 55, 227560))])
1d18365735eec1823e29c1a2fd0ffb346358da107f3f1db434e036c7d6f7ce8c5f2d0946b0004f8ff09871b809f1a3ef960731770f7f1bb85a4778a63d38243e5aedc21c84994ed2503399cff3d40adbeb720efd0ccbe3f60737aa35eface6cf14c6b84e9fa8f476c21d64186fb666b2476

In [6]:
# Next section to start https://www.tutorialspoint.com/python_blockchain/python_blockchain_creating_multiple_transactions.htm

In [7]:
def display_transaction(transaction):
   #for transaction in transactions:
   dict = transaction.to_dict()
   print ("sender: " + dict['sender'])
   print ('-----')
   print ("recipient: " + dict['recipient'])
   print ('-----')
   print ("value: " + str(dict['value']))
   print ('-----')
   print ("time: " + str(dict['time']))
   print ('-----')

In [8]:
# Queue Where transactions are stored before they are added to the blockchain
# Not implementing the queue management logic
transactions = []

In [9]:
# Creating multiple clients
mike = Client()
steve = Client()
rose = Client()
rachel = Client()

In [10]:
# Initiating the first transaction
t1 = Transaction(
    mike,
    steve.identity,
    15.0
)

In [11]:
# Adding the first transaction to the queue
t1.sign_transaction()
transactions.append(t1)

In [12]:
t2 = Transaction(
   mike,
   rose.identity,
   6.0
)
t2.sign_transaction()
transactions.append(t2)
t3 = Transaction(
   steve,
   rachel.identity,
   2.0
)
t3.sign_transaction()
transactions.append(t3)
t4 = Transaction(
   rose,
   steve.identity,
   4.0
)
t4.sign_transaction()
transactions.append(t4)
t5 = Transaction(
   mike,
   rose.identity,
   7.0
)
t5.sign_transaction()
transactions.append(t5)
t6 = Transaction(
   rose,
   rachel.identity,
   3.0
)
t6.sign_transaction()
transactions.append(t6)
t7 = Transaction(
   rachel,
   steve.identity,
   8.0
)
t7.sign_transaction()
transactions.append(t7)
t8 = Transaction(
   rachel,
   mike.identity,
   1.0
)
t8.sign_transaction()
transactions.append(t8)
t9 = Transaction(
   steve,
   mike.identity,
   5.0
)
t9.sign_transaction()
transactions.append(t9)
t10 = Transaction(
   steve,
   rose.identity,
   3.0
)
t10.sign_transaction()
transactions.append(t10)

In [13]:
for transaction in transactions:
    display_transaction (transaction)
    print('----------------')

sender: 30819f300d06092a864886f70d010101050003818d0030818902818100c49b29192ed51067c9e1166f10c44072c89c2cffe2004092747971a13c0096c2f001d075a685497ad4abaf8a334692f140d9d2abcefb1989fb1c22ae5942ae4a2ccc1fe1200e226c8a108589edb40ab0d0e0dc280044fef3db95d55efbece2166b678c3ef8067572badb32f3652e0b08b3c98a0185604e67e1b4cbcd253b20630203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100949e27a6c2001e17fa8d6afabb4398cf43711b44261b0520680f9b6d0810aa61028d168faf1538647526e8743306108c78b7d207ef3ba0390d444a06bef3a9c0497a79f4294501430a11aa95ccd89779a79b7ddcad9c9c089dc363a18c66c83b0f9f36635c8f9a17d6cdc1e42a607f9bcc595a1e0995ce9c2ee2cbc29f8693790203010001
-----
value: 15.0
-----
time: 2021-02-04 04:20:56.396418
-----
----------------
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100c49b29192ed51067c9e1166f10c44072c89c2cffe2004092747971a13c0096c2f001d075a685497ad4abaf8a334692f140d9d2abcefb1989fb1c22ae5942ae4a2ccc1fe1200e226c8a108589edb40ab0d0e0dc280044fef3db9

A block consists of a varying number of transactions. For simplicity, in our case we will assume that the block consists of a fixed number of transactions

In [14]:
# blockchain is a list of blocks chained to each other
# For our list we will create the following list variable
TPCoins = []

class Block:
    '''
    Blockchain
    ...

    Attributes
    -----------
    verified_transactions : list
        Verified transactions will be added to this block. Each block also contains the
        hash value of the previous block. Making the chain immutable
    previous_block_hash : str
        Instance varaible to store the previous hash
    Nonce : str
        Variable to store the Nonce created by the miner during the mining process
    '''
    def __init__(self):
        self.verified_transactions = []
        self.previous_block_hash = ''
        self.Nonce = ''

def dump_blockchain(self):
    '''
    Dumps the contents of the entire blockchain.

    Description
    ------------
    1. It prints the number of blocks in the current blockchain
    2. Next the blocks are copied to a temp variable named block_temp
    3. Print block number as heading for each block
    4. Then iterate over the transactions in each block
    '''
    print('Number of blocks in the chain: ' + str(len (self)))

    for x in range (len(TPCoins)):
        block_temp = TPCoins[x]
        print('block # ' + str(x))
        for transaction in block_temp.verified_transactions:
            display_transaction (transaction)
            print ('--------------')
        print ('=====================================')

# Global variable 
last_block_hash = ''

In [15]:
# Genesis block: 500 coins are given out to the client michael
michael = Client()

t0 = Transaction (
    'Genesis',
    michael.identity,
    500.00
)

# Create an instance of the Block class
block0 = Block()

# Where this is the first block we initialize the previous_block_hash and Nonce to none.
block0.previous_block_hash = None 
block0.Nonce = None

# Add the transaction to the verified transactions
block0.verified_transactions.append (t0)

# The block is now ready to be added to the blockchain. Before we do this we will hash the
# block and store its value in the global variable `last_block_hash`
digest = hash (block0)
last_block_hash = digest

In [16]:

TPCoins.append(block0)

dump_blockchain(TPCoins)


Number of blocks in the chain: 1
block # 0
sender: Genesis
-----
recipient: 30819f300d06092a864886f70d010101050003818d00308189028181008760f10628d8d0fbe2dc4e92b6777301e3f2ec965804d14e3b9cfa0eefc3d23cd2e2bc8d0d93d9c82b9f92c0b3709ed49a9238316ecfcbe9fd6c5da9554e61b77d6e5ba8db8923af6d176b716490b8242274a75cb3fa6aa2ab1c38cf3063c0f6ce65c0bd1eb72be52de808ad987f7b1abc449598b3ff54157f2a216df5366f570203010001
-----
value: 500.0
-----
time: 2021-02-04 04:20:57.285985
-----
--------------


In [23]:
def sha256(message):
    '''
    Utility function for creating a digest on a given message

    This function takes a message as it parameter, encodes it to ASCII, generates a
    hexadecimal digest and returns the value to the caller.
    '''
    return hashlib.sha256(message.encode('ascii')).hexdigest()



# Mining Strategy: Generate a hash on the given message prefixed with a given number of 1's
# The given number of 1's is specified as a parameter to mine function specified as the 
# difficulty level.

# E.g. if you specify a difficulty level of 2, the gnerate has on a given message will start
# with two 1's.

def mine(message, difficulty=1):
    '''
    Mining function to generate a hash on the given message prefixed with 1's

    Parameters:
    ------------
    message : str

    difficulty : int
        difficulty level that specifies how many ones to add to the beginning of the message
    '''
    assert difficulty >= 1

    prefix = '1' * difficulty
    print(prefix)
    for i in range(1000):
        digest = sha256(str(hash(message)) + str(i))
        if digest.startswith(prefix):
            print('after ' + str(i) + ' iterations found nonce: '+ digest)
            return digest



In [24]:
# test mining function
mine('test message', 2)
# Not returing correctly

11
after 192 iterations found nonce: 110ee90ae717aa331a76aab5fc5ac5125a772da9a88533ac6a981ad5ee0bbbaf


'110ee90ae717aa331a76aab5fc5ac5125a772da9a88533ac6a981ad5ee0bbbaf'

In [25]:
# Global variable to track the number of messages already mined
last_transaction_index = 0

In [26]:
# Add a new block to the blockchain
block = Block()

for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    # validate transaction - Not added as part of this tutorial

    # Essentially you would validate the hash generated by the sender against the hash
    # generated by the miner using the senders public key.
    # Then the miner would verify the sender has sufficient balance to pay the current
    # transaction.

    # if valid
    block.verified_transactions.append(temp_transaction)
   last_transaction_index += 1

block.previous_block_hash = last_block_hash
block.Nonce = mine (block, 2)
digest = hash (block)
TPCoins.append (block)
last_block_hash = digest

11
after 149 iterations found nonce: 11b3bb6b1164df5eb4121645e75e278e738f06009519efb21e9e31d345f4b9da
11
after 149 iterations found nonce: 11b3bb6b1164df5eb4121645e75e278e738f06009519efb21e9e31d345f4b9da
11
after 149 iterations found nonce: 11b3bb6b1164df5eb4121645e75e278e738f06009519efb21e9e31d345f4b9da


In [28]:
# Code for adding the next two blocks
# Miner 2 adds a block
block = Block()

for i in range(3):
   temp_transaction = transactions[last_transaction_index]
   # validate transaction
   # if valid
   block.verified_transactions.append (temp_transaction)
   last_transaction_index += 1
block.previous_block_hash = last_block_hash
block.Nonce = mine (block, 2)
digest = hash (block)
TPCoins.append (block)
last_block_hash = digest
# Miner 3 adds a block
block = Block()

for i in range(3):
   temp_transaction = transactions[last_transaction_index]
   #display_transaction (temp_transaction)
   # validate transaction
   # if valid
   block.verified_transactions.append (temp_transaction)
   last_transaction_index += 1

block.previous_block_hash = last_block_hash
block.Nonce = mine (block, 2)
digest = hash (block)

TPCoins.append (block)
last_block_hash = digest


11
after 24 iterations found nonce: 1198476a0dfb4c5aadea21ca806a284da9f46cd349fd2c4fc13a86b03795780e
11
after 349 iterations found nonce: 118e72c5ef576ffb071dda9a5a27d5c2b80e2fa9d60043a1313c45260a884b74


In [30]:
# Reviewing blockchain
dump_blockchain(TPCoins)

Number of blocks in the chain: 6
block # 0
sender: Genesis
-----
recipient: 30819f300d06092a864886f70d010101050003818d00308189028181008760f10628d8d0fbe2dc4e92b6777301e3f2ec965804d14e3b9cfa0eefc3d23cd2e2bc8d0d93d9c82b9f92c0b3709ed49a9238316ecfcbe9fd6c5da9554e61b77d6e5ba8db8923af6d176b716490b8242274a75cb3fa6aa2ab1c38cf3063c0f6ce65c0bd1eb72be52de808ad987f7b1abc449598b3ff54157f2a216df5366f570203010001
-----
value: 500.0
-----
time: 2021-02-04 04:20:57.285985
-----
--------------
block # 1
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100c49b29192ed51067c9e1166f10c44072c89c2cffe2004092747971a13c0096c2f001d075a685497ad4abaf8a334692f140d9d2abcefb1989fb1c22ae5942ae4a2ccc1fe1200e226c8a108589edb40ab0d0e0dc280044fef3db95d55efbece2166b678c3ef8067572badb32f3652e0b08b3c98a0185604e67e1b4cbcd253b20630203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100949e27a6c2001e17fa8d6afabb4398cf43711b44261b0520680f9b6d0810aa61028d168faf1538647526e8743306108c78b7d2